# Process proteomics data for Bayesian MCA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cobra, gzip, re

## Get genes from model

In [3]:
# model needs to updated for round 2 to include ethanol, erythritol, and citric acid secretion
model = cobra.io.load_json_model(
    "../../models/iJB1325_HP.nonnative_genes.pubchem.flipped.nonzero.reduced.json"
)
len(model.metabolites)
# 

171

In [6]:
model.boundary

[<Reaction BOUNDARY_ACe at 0x7fec245957c0>,
 <Reaction BOUNDARY_CO2e at 0x7fec245c4790>,
 <Reaction BOUNDARY_FORe at 0x7fec245cf730>,
 <Reaction BOUNDARY_GLCe at 0x7fec245cfa60>,
 <Reaction BOUNDARY_GLCNTe at 0x7fec245d7dc0>,
 <Reaction BOUNDARY_He at 0x7fec245dbdf0>,
 <Reaction BOUNDARY_H2Oe at 0x7fec245dbd90>,
 <Reaction BOUNDARY_HNO3e at 0x7fec245df520>,
 <Reaction BOUNDARY_O2e at 0x7fec245e7be0>,
 <Reaction BOUNDARY_TARe at 0x7fec245e7d90>,
 <Reaction BOUNDARY_SMPYRKe at 0x7fec25714820>,
 <Reaction EX_3hpp_e at 0x7fec25754ee0>]

# transcript to protein map for Aspni7



In [7]:
prot_gene_re = re.compile(r"proteinId=(\d+);.*transcriptId=(\d+)")

In [10]:
# this maps transcript IDs to protein ID 
prot_gene_map, gene_prot_map = {}, {}
prot_gene_re = re.compile(r"proteinId=(\d+);.*transcriptId=(\d+)")
with gzip.open("../../data/round1/Aspni7_FilteredModels1_deflines.gff3.gz", "rt") as gff3:
    for line in gff3:
        m = prot_gene_re.search(line)
        # print(line)
        if m:
            gene_prot_map[m.group(2)] = m.group(1)
            prot_gene_map[m.group(1)] = m.group(2)
len(gene_prot_map)

11910

In [21]:
print(len([g for g in model.genes if g.id in gene_prot_map]))
print(len([g for g in model.genes if g.id not in gene_prot_map]))

1317
11


In [23]:
[g for g in model.genes if g.id not in gene_prot_map]

[<Gene 1097533 at 0x7fec1c3a2220>,
 <Gene 1090687 at 0x7fec1c3da190>,
 <Gene 1123432 at 0x7fec1c3da250>,
 <Gene Unknown5 at 0x7fec1c3dad30>,
 <Gene 1070139 at 0x7fec1c3e5460>,
 <Gene 205484 at 0x7fec1c3fc040>,
 <Gene 1092679 at 0x7fec1c4071f0>,
 <Gene 1172305 at 0x7fec1c407610>,
 <Gene PAND_Tribolium_castaneum at 0x7fec1c429220>,
 <Gene HPDH_escherichia_coli at 0x7fec1c429250>,
 <Gene BAPAT_Bacillus_cereus at 0x7fec1c429280>]

## Read round 2 data

In [25]:
data_file = '../../data/round2/ABF_Aniger_BMCA_2_DATA.xlsx'

In [26]:
pd.ExcelFile(data_file).sheet_names

['notes',
 'Extracellular data summary',
 'Multiomics metadata',
 'Extracellular_metabolite',
 'global_protein',
 'targeted_protein',
 'intracellular_metabolite']

In [31]:
pd.read_excel(data_file, sheet_name='Extracellular data summary', index_col=[0,1,2], header=[0,1,2])

averages of the 3 replicates  \
ICE_ID                                                                     Unnamed: 3_level_1   
                                                                                        t = 0   
1  wild-type                                          ABF_008340                          120   
2  (βAl-3HP)+                                         ABF_008343                          120   
3  (βAl-3HP)+, pyc2+                                  ABF_008345                          120   
4  (βAl-3HP)+, pyc2+, Δald6a                          ABF_008348                          120   
5  (βAl-3HP)+, pyc2+, Δald6a, aat2(cy)+               ABF_011231                          120   
6  (βAl-3HP)+, pyc2+, Δald6a, aat2(mt)+               ABF_011232                          120   
7  (βAl-3HP)+, pyc2++, Δald6a                         ABF_011233                          120   
8  (βAl-3HP)+, pyc2+, Δald6a, mdhA+                   ABF_011234                          120   
9  (βAl-3HP)+, pyc2+, Δald6a, purU+                   ABF_011236                          120   
10 (βAl-3HP)+, pyc2+, Δald6a, Δadh1                   ABF_011239                          120   
11 (βAl-3HP)+, pyc2+, Δald6a, ΔiscA                   ABF_011240                          120   
12 (βAl-3HP)+, pyc2+, Δald6a, ΔiscB                   ABF_011241                          120   
13 (βAl-3HP)+, pyc2+, Δald6a, ΔscotA                  ABF_011242                          120   
14 (βAl-3HP)+, pyc2+, Δald6a, Δech3                   ABF_011243                          120   
15 (βAl-3HP)+, pyc2+, Δald6a, ΔcoaT                   ABF_011244                          120   
16 (βAl-3HP)++, pyc2++, Δald6a                        ABF_010216                          120   
17 (βAl-3HP)++, pyc2++, Δald6a+NeoR                   ABF_011245                          120   
18 (βAl-3HP)++, pyc2++, Δald6a, ΔiscA, ΔoahA-C5       ABF_011249                          120   
19 (βAl-3HP)++, pyc2++, Δald6a, ΔiscA, ΔoahA-C7       ABF_011250                          120   
20 (βAl-3HP)++, pyc2++, Δald6a, ΔiscA, ΔoahA, Δald3   ABF_011253                          120   
21 (βAl-3HP)++, pyc2++, Δald6a, ΔiscA, ΔoahA, Δald... ABF_011254                          120   
22 (βAl-3HP)++, pyc2++, Δald6a, ΔiscA, ΔoahA, Dald... ABF_011255                          120   
23 (βAl-3HP)++, pyc2++, Δald6a, ΔiscA, ΔoahA, Dald... ABF_011256                          120   
24 (βAl-3HP)++, pyc2+, Δald6a                         ABF_010214                          120   

                                                                                 \
ICE_ID                                                           Glucose  (g/L)   
                                                                         t = 92   
1  wild-type                                          ABF_008340      47.422211   
2  (βAl-3HP)+                                         ABF_008343      49.789625   
3  (βAl-3HP)+, pyc2+                                  ABF_008345      50.478728   
4  (βAl-3HP)+, pyc2+, Δald6a                          ABF_008348      50.585983   
5  (βAl-3HP)+, pyc2+, Δald6a, aat2(cy)+               ABF_011231      57.240266   
6  (βAl-3HP)+, pyc2+, Δald6a, aat2(mt)+               ABF_011232      49.985311   
7  (βAl-3HP)+, pyc2++, Δald6a                         ABF_011233      46.198211   
8  (βAl-3HP)+, pyc2+, Δald6a, mdhA+                   ABF_011234      46.368167   
9  (βAl-3HP)+, pyc2+, Δald6a, purU+                   ABF_011236      47.643453   
10 (βAl-3HP)+, pyc2+, Δald6a, Δadh1                   ABF_011239      47.875584   
11 (βAl-3HP)+, pyc2+, Δald6a, ΔiscA                   ABF_011240      53.007228   
12 (βAl-3HP)+, pyc2+, Δald6a, ΔiscB                   ABF_011241      53.819160   
13 (βAl-3HP)+, pyc2+, Δald6a, ΔscotA                  ABF_011242      46.728980   
14 (βAl-3HP)+, pyc2+, Δald6a, Δech3                   ABF_011243      48.104758   
15 (βAl-3HP)+, pyc2+, Δald6a, Δcoa

## Reference strain SF ABF180_17_R2

**Best performing strain: [(βAl-3HP)++, pyc2++, Δald6a+NeoR] (Strain ABF-10216R8, ICE ID ABF_011245)**

In [113]:
reference_strain = "SF ABF180_17_R2"

# Global proteomics at 92 hours

In [94]:
global_proteomics = pd.read_excel(data_file, sheet_name='global_protein')
global_proteomics.head()

,Protein,Redundancy,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
0,jgi|Aspni7|1169809,1,32.176158,33.153051,33.028620,32.895799,33.449526,33.073643,33.113237,32.740618,...,33.138326,33.273066,32.658036,33.370635,33.284373,33.477577,33.231232,32.414363,32.799510,32.672570
1,jgi|Aspni7|1141219,1,31.468430,31.376778,31.415586,31.452133,31.314783,31.380998,32.073355,31.353739,...,31.404182,31.683614,31.175667,31.716532,31.392035,31.774963,31.876349,31.544927,31.568684,31.174506
2,jgi|Aspni7|201546,1,31.821116,31.846684,31.613288,31.870536,32.422308,32.420785,32.220315,31.869863,...,32.118247,32.307963,31.825711,32.264260,32.544977,32.375467,32.288612,31.902450,32.053782,31.860052
3,jgi|Aspni7|1147622,1,33.517377,33.677164,33.641382,34.362115,34.341117,34.455973,33.899787,33.849046,...,33.894014,33.701087,33.921813,33.806313,33.762136,34.156727,34.426063,33.736694,33.530660,33.751044
4,jgi|Aspni7|1142593,1,34.154545,34.126483,34.117472,34.047973,33.851506,33.671474,34.104701,33.881193,...,33.552360,33.609413,33.737357,33.676070,33.590278,33.876611,33.284638,33.554197,33.490469,33.466278


### Remove redundant rows

In [95]:
print(len(global_proteomics))
print(len(global_proteomics[global_proteomics['Redundancy'] == 1]))

4585
4155


In [99]:
print(len(global_proteomics.Protein.unique()))
print(len(global_proteomics.drop(columns='Redundancy').drop_duplicates()))

4233
4233


In [101]:
global_proteomics = global_proteomics.drop(columns='Redundancy').drop_duplicates()
global_proteomics.head()

,Protein,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
0,jgi|Aspni7|1169809,32.176158,33.153051,33.028620,32.895799,33.449526,33.073643,33.113237,32.740618,32.768240,...,33.138326,33.273066,32.658036,33.370635,33.284373,33.477577,33.231232,32.414363,32.799510,32.672570
1,jgi|Aspni7|1141219,31.468430,31.376778,31.415586,31.452133,31.314783,31.380998,32.073355,31.353739,31.441164,...,31.404182,31.683614,31.175667,31.716532,31.392035,31.774963,31.876349,31.544927,31.568684,31.174506
2,jgi|Aspni7|201546,31.821116,31.846684,31.613288,31.870536,32.422308,32.420785,32.220315,31.869863,32.164412,...,32.118247,32.307963,31.825711,32.264260,32.544977,32.375467,32.288612,31.902450,32.053782,31.860052
3,jgi|Aspni7|1147622,33.517377,33.677164,33.641382,34.362115,34.341117,34.455973,33.899787,33.849046,33.773548,...,33.894014,33.701087,33.921813,33.806313,33.762136,34.156727,34.426063,33.736694,33.530660,33.751044
4,jgi|Aspni7|1142593,34.154545,34.126483,34.117472,34.047973,33.851506,33.671474,34.104701,33.881193,34.417227,...,33.552360,33.609413,33.737357,33.676070,33.590278,33.876611,33.284638,33.554197,33.490469,33.466278


### Map protein IDs to transcript IDs in model

In [109]:
for x in global_proteomics.Protein:
    if x.split('|')[-1] not in prot_gene_map:
        print(x, x in model.genes)

HPDH_escherichia_coli True
BAPAT_Bacillus_cereus True
PAND_Tribolium_castaneum True


In [111]:
global_proteomics.index = [prot_gene_map[x.split('|')[-1]] if x.split('|')[-1] in prot_gene_map
                           else x for x in global_proteomics.Protein]
global_proteomics.index.name = 'Transcript'
global_proteomics = global_proteomics.drop(columns='Protein')
global_proteomics.head()

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
Transcript,,,,,,,,,,,,,,,,,,,,,
1170085,32.176158,33.153051,33.028620,32.895799,33.449526,33.073643,33.113237,32.740618,32.768240,32.839315,...,33.138326,33.273066,32.658036,33.370635,33.284373,33.477577,33.231232,32.414363,32.799510,32.672570
1141495,31.468430,31.376778,31.415586,31.452133,31.314783,31.380998,32.073355,31.353739,31.441164,31.516954,...,31.404182,31.683614,31.175667,31.716532,31.392035,31.774963,31.876349,31.544927,31.568684,31.174506
201546,31.821116,31.846684,31.613288,31.870536,32.422308,32.420785,32.220315,31.869863,32.164412,32.174083,...,32.118247,32.307963,31.825711,32.264260,32.544977,32.375467,32.288612,31.902450,32.053782,31.860052
1147898,33.517377,33.677164,33.641382,34.362115,34.341117,34.455973,33.899787,33.849046,33.773548,33.887928,...,33.894014,33.701087,33.921813,33.806313,33.762136,34.156727,34.426063,33.736694,33.530660,33.751044
1142869,34.154545,34.126483,34.117472,34.047973,33.851506,33.671474,34.104701,33.881193,34.417227,33.874547,...,33.552360,33.609413,33.737357,33.676070,33.590278,33.876611,33.284638,33.554197,33.490469,33.466278


### Normalize data to the reference strain

In [114]:
normalized_global_proteomics = global_proteomics.divide(
    global_proteomics[reference_strain], axis=0).replace(-np.inf, 1).replace(np.inf,1).fillna(1)
normalized_global_proteomics.head()

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
Transcript,,,,,,,,,,,,,,,,,,,,,
1170085,0.967266,0.996633,0.992893,0.988900,1.005546,0.994246,0.995436,0.984235,0.985065,0.987202,...,0.996191,1.000241,0.981752,1.003174,1.000581,1.006389,0.998983,0.974427,0.986005,0.982189
1141495,1.012991,1.010041,1.011290,1.012467,1.008045,1.010177,1.032464,1.009299,1.012114,1.014553,...,1.010923,1.019918,1.003567,1.020978,1.010532,1.022859,1.026123,1.015454,1.016219,1.003530
201546,0.979618,0.980405,0.973220,0.981140,0.998126,0.998079,0.991908,0.981119,0.990187,0.990484,...,0.988765,0.994606,0.979760,0.993260,1.001902,0.996684,0.994010,0.982122,0.986781,0.980817
1147898,0.995988,1.000736,0.999673,1.021090,1.020466,1.023879,1.007351,1.005843,1.003600,1.006999,...,1.007180,1.001447,1.008006,1.004574,1.003261,1.014986,1.022990,1.002505,0.996382,1.002931
1142869,1.018378,1.017542,1.017273,1.015201,1.009343,1.003975,1.016892,1.010228,1.026211,1.010030,...,1.000423,1.002124,1.005939,1.004112,1.001554,1.010091,0.992440,1.000478,0.998578,0.997856


In [115]:
normalized_global_proteomics.to_csv('../../data/round2/normalized_global_proteomics.csv')

# Targeted proteomics at 92 hours

In [116]:
targeted_proteomics = pd.read_excel(data_file, sheet_name='targeted_protein')
targeted_proteomics.head()

,Protein,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
0,jgi|Aspni7|1145331,0.863964,0.862520,0.784762,1.387217,1.222693,1.338794,2.253985,2.312679,2.219523,...,1.949292,1.939182,2.000392,2.072654,1.588859,1.738713,1.701155,2.279920,2.163910,2.221940
1,jgi|Aspni7|1136064,-0.365720,-0.177613,-0.216719,-0.697055,-0.819734,-0.585171,-1.308847,-1.083605,-1.192970,...,-1.261217,-1.341484,-1.355311,-1.309444,-0.832876,-0.755307,-0.674192,-1.034830,-1.133321,-1.127115
2,jgi|Aspni7|1099056,-6.522509,-6.291478,-6.417873,-6.335275,-6.450880,-6.323233,-6.591499,-6.581977,-6.606718,...,-6.559411,-6.516137,-6.610014,-6.661304,-6.809476,-6.743531,-6.711409,-6.644841,-6.440649,-6.430682
3,jgi|Aspni7|1128408,-0.423381,-0.402166,-0.347003,-1.168125,-1.355903,-1.260792,-1.863573,-1.630895,-1.810275,...,-1.945666,-1.964705,-2.096481,-2.029213,-1.790707,-1.846235,-1.801093,-2.114836,-2.029731,-2.032081
4,pand,-4.342076,-5.353449,-4.744258,4.954097,4.760225,5.051498,5.349948,5.566743,5.271986,...,5.600039,5.312262,5.209465,5.458940,4.827779,5.020662,4.765847,5.724487,5.634604,5.672021


### Map protein IDs to transcript IDs in model

In [117]:
for x in targeted_proteomics.Protein:
    if x.split('|')[-1] not in prot_gene_map:
        print(x, x in model.genes)

pand False
bapat False
hpdh False


In [118]:
prot_gene_map['pand'] = 'PAND_Tribolium_castaneum'
prot_gene_map['bapat'] = 'BAPAT_Bacillus_cereus'
prot_gene_map['hpdh'] = 'HPDH_escherichia_coli'

In [119]:
targeted_proteomics.index = [prot_gene_map[x.split('|')[-1]] if x.split('|')[-1] in prot_gene_map
                           else x for x in targeted_proteomics.Protein]
targeted_proteomics.index.name = 'Transcript'
targeted_proteomics = targeted_proteomics.drop(columns='Protein')
targeted_proteomics.head()

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
Transcript,,,,,,,,,,,,,,,,,,,,,
1145607,0.863964,0.862520,0.784762,1.387217,1.222693,1.338794,2.253985,2.312679,2.219523,2.254670,...,1.949292,1.939182,2.000392,2.072654,1.588859,1.738713,1.701155,2.279920,2.163910,2.221940
1136340,-0.365720,-0.177613,-0.216719,-0.697055,-0.819734,-0.585171,-1.308847,-1.083605,-1.192970,-1.195760,...,-1.261217,-1.341484,-1.355311,-1.309444,-0.832876,-0.755307,-0.674192,-1.034830,-1.133321,-1.127115
1099332,-6.522509,-6.291478,-6.417873,-6.335275,-6.450880,-6.323233,-6.591499,-6.581977,-6.606718,-6.704133,...,-6.559411,-6.516137,-6.610014,-6.661304,-6.809476,-6.743531,-6.711409,-6.644841,-6.440649,-6.430682
1128684,-0.423381,-0.402166,-0.347003,-1.168125,-1.355903,-1.260792,-1.863573,-1.630895,-1.810275,-1.762544,...,-1.945666,-1.964705,-2.096481,-2.029213,-1.790707,-1.846235,-1.801093,-2.114836,-2.029731,-2.032081
PAND_Tribolium_castaneum,-4.342076,-5.353449,-4.744258,4.954097,4.760225,5.051498,5.349948,5.566743,5.271986,5.360159,...,5.600039,5.312262,5.209465,5.458940,4.827779,5.020662,4.765847,5.724487,5.634604,5.672021


### Normalize data to the reference strain

In [121]:
normalized_targeted_proteomics = targeted_proteomics.divide(
    targeted_proteomics[reference_strain], axis=0).replace(-np.inf, 1).replace(np.inf,1).fillna(1)
normalized_targeted_proteomics.head()

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
Transcript,,,,,,,,,,,,,,,,,,,,,
1145607,0.460337,0.459568,0.418137,0.739137,0.651475,0.713336,1.200969,1.232242,1.182606,1.201334,...,1.038622,1.033236,1.065849,1.104352,0.846576,0.926422,0.906410,1.214787,1.152975,1.183895
1136340,0.320153,0.155484,0.189717,0.610205,0.717599,0.512261,1.145771,0.948593,1.044332,1.046773,...,1.104075,1.174341,1.186445,1.146293,0.729103,0.661199,0.590191,0.905895,0.992115,0.986682
1099332,0.932467,0.899438,0.917508,0.905699,0.922226,0.903978,0.942329,0.940968,0.944505,0.958432,...,0.937742,0.931556,0.944976,0.952309,0.973492,0.964064,0.959472,0.949955,0.920764,0.919339
1128684,0.209681,0.199174,0.171855,0.578518,0.671516,0.624412,0.922942,0.807707,0.896546,0.872907,...,0.963598,0.973028,1.038290,1.004976,0.886855,0.914355,0.891998,1.047381,1.005232,1.006396
PAND_Tribolium_castaneum,-0.776427,-0.957275,-0.848343,0.885865,0.851198,0.903282,0.956649,0.995415,0.942708,0.958475,...,1.001369,0.949910,0.931529,0.976138,0.863278,0.897768,0.852203,1.023622,1.007550,1.014240


In [127]:
normalized_targeted_proteomics.to_csv('../../data/round2/normalized_targeted_proteomics.csv')

### Calculate enzyme activity from normalized targeted proteomics

In [120]:
def get_enzyme_activity_expression(proteins, model):
    """Get enzyme activity expression as a function of protein expression

    Take the min over all subunits for each isoenzyme and sum over all isoenzymes

    :param proteins:  protein x experiment dataframe
    :param model:  cobra model
    :returns:  reaction x experiment dataframe
    """
    enzyme_expression = {}
    transcripts = list(proteins.index)
    for r in model.reactions:
        if r.gene_reaction_rule and (
            len(set([g.id for g in r.genes]) & set(transcripts)) > 0
        ):
            subunits_expression = {}
            for x in [x.strip("() ") for x in r.gene_reaction_rule.split(" or ")]:
                # Take the min over all subunits for each study line
                subunits = [
                    y.strip("() ")
                    for y in x.split(" and ")
                    if y.strip("() ") in transcripts
                ]
                if len(subunits) > 0:
                    subunits_expression[x] = proteins.loc[subunits].min(axis=0)
            enzyme_expression[r.id] = pd.DataFrame(subunits_expression).sum(axis=1)
    enzyme_expression = pd.DataFrame(enzyme_expression).T
    enzyme_expression.index.name = "rxn"
    return enzyme_expression

In [129]:
normalized_rxns_targeted_prot = get_enzyme_activity_expression(
    normalized_targeted_proteomics, model
)
normalized_rxns_targeted_prot.to_csv("../../data/round2/normalized_targeted_enzyme_activities.csv")
normalized_rxns_targeted_prot.head()

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
rxn,,,,,,,,,,,,,,,,,,,,,
r5a,-0.395364,-0.376567,-0.576585,0.154199,0.505214,0.224515,0.381607,0.053327,0.478963,0.309413,...,0.372007,0.377086,0.529102,0.281917,0.770407,0.755794,0.761661,0.534447,0.571007,0.581337
r7,-11.987266,-15.088876,-17.233646,-11.921451,-3.612668,-7.824882,-5.386092,-16.795537,-11.027185,-10.599159,...,5.358484,3.758266,5.215541,2.467729,8.037332,2.150435,4.249706,-3.141180,-1.305362,-2.431933
r8,-11.987266,-15.088876,-17.233646,-11.921451,-3.612668,-7.824882,-5.386092,-16.795537,-11.027185,-10.599159,...,5.358484,3.758266,5.215541,2.467729,8.037332,2.150435,4.249706,-3.141180,-1.305362,-2.431933
r10,0.209681,0.199174,0.171855,0.578518,0.671516,0.624412,0.922942,0.807707,0.896546,0.872907,...,0.963598,0.973028,1.038290,1.004976,0.886855,0.914355,0.891998,1.047381,1.005232,1.006396
r12a,1.628446,1.727046,1.712109,1.249154,1.185953,1.229736,1.002695,1.128017,1.015044,1.052283,...,1.018388,1.072428,1.026140,1.085602,1.096311,1.080677,1.141810,0.989098,0.929171,0.958589
